__This Is The Final One__

In [1]:
import open3d as o3d
import numpy as np
#Directly use point cloud...
#【Load point cloud data】 (指是為了顯示方便而已, 實際是用mesh去算)
ply_file_path = ("../PointCloud/TomatoPlant_size_modified_only1tomato.ply")
pcd = o3d.io.read_point_cloud(ply_file_path)
pcd.paint_uniform_color([0, 0.5, 0.5]) #把point cloud data統一是綠色

#【Load mesh data】
mesh0 = o3d.io.read_triangle_mesh(ply_file_path)
mesh = o3d.t.geometry.TriangleMesh.from_legacy(mesh0)

#【求mesh 的 中心】(變成[center_x, center_y, center_z] 的形式)
mesh_center = mesh.get_center()
print(mesh_center)
# Extract the individual scalar values from the Tensor
center_x = mesh_center[0].item() #<Note>.item(): 是因為mesh.get_center()是抓到tensor
center_y = mesh_center[1].item()
center_z = mesh_center[2].item()
# Create the center list
center = [center_x, center_y, center_z]

print("center:",center)

# 【Create a scene and add the triangle mesh】把mesh放進ray scan的空間，為了下面的ray scan
scene = o3d.t.geometry.RaycastingScene()
_ = scene.add_triangles(mesh)  # we do not need the geometry ID for mesh

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
[0.181869 -0.146326 0.484344]
Tensor[shape={3}, stride={1}, Float32, CPU:0, 0x18fbcac84a0]
center: [0.18186891078948975, -0.14632552862167358, 0.4843443036079407]


In [2]:
#【建立sample point】
# [a. 建立各方向的sample point]
# N = 10000
# query_points = np.random.uniform(low=min_bound, high=max_bound,
                                 # size=[N, 3]).astype(np.float32)
# [b. 建立球型的sample point]
# center = np.array([0.0, 0.0, 0.0])  # Adjust the center as needed
radius = 0.2  # Adjust the radius as needed
N = 5000  # Number of points to generate

# Generate random points in the z-plane within a specific radius
phi = np.random.uniform(0, 2 * np.pi, N)  # Azimuthal angle
# costheta = np.random.uniform(-1, 1, N)  # Cosine of the polar angle (想要整圈, 想要上半圈就 0~1) 
costheta = 0  # 如果是只想要再z平面的話
theta = np.arccos(costheta)  # Polar angle
r = np.random.uniform(0, radius, N)
# Convert spherical coordinates to Cartesian coordinates
x = center[0] + r * np.sin(theta) * np.cos(phi)
y = center[1] + r * np.sin(theta) * np.sin(phi)
z = center[2] + r * np.cos(theta)

#想要查SDF value的那個點(query_points要裝[[a,b,c]]的nest list 型式去裝點)
query_points = np.stack([x, y, z], axis=-1).astype(np.float32)
# query_points = o3d.core.Tensor([[16,16,16]], dtype=o3d.core.Dtype.Float32)
print(query_points.shape)


# Initialize empty lists for points and colors
points = []
colors = []

def add_point(point, color):
    global points, colors
    points.append(point)
    colors.append(color)

#跑過每個要查的點，如果SDF<=0代表在mesh內側，就要顯示乘紅色(之後就是就要information gain 為1)
for point_q in query_points:
    point_q_formed=[[point_q]]
    signed_distance = scene.compute_signed_distance(point_q_formed)
    # print("query_points: ", query_points)
    occupancy = scene.compute_occupancy(point_q_formed)
    
    if (signed_distance <= 0): #表示在裡面
        add_point(point_q, [1, 0, 0])  # Red color
        # print(signed_distance)
    else: 
        add_point(point_q, [0, 0, 1])
    

(5000, 3)


In [3]:
import numpy as np

query_point_vis = o3d.geometry.PointCloud()
# query_point_vis.points = o3d.utility.Vector3dVector(np.array([[0, 0, 0]]))
query_point_vis.points = o3d.utility.Vector3dVector(np.array(points))
query_point_vis.colors = o3d.utility.Vector3dVector(np.array(colors))

coordinate_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(
    size=0.1, origin=center
)
# Visualize the mesh and the query point together
# o3d.visualization.draw_geometries([pcd, query_point_vis])#, #, coordinate_frame
#                                   # zoom=0.7,
#                                   # front=[0.0, 0.0, -1.0],
#                                   # lookat=[0, 0, 0],
#                                   # up=[0.0, 1.0, 0.0])


image_width = 640  # in pixels
image_height = 480  # in pixels
vis = o3d.visualization.Visualizer()
vis.create_window(window_name='SDF Value and Sample point', width=image_width, height=image_height) 
# vis.create_window()
# vis.add_geometry(mesh0)
vis.add_geometry(query_point_vis)
vis.add_geometry(pcd)

#【Note: 那個相機view的符號】
# standardCameraParametersObj     = vis.get_view_control().convert_to_pinhole_camera_parameters()
# cameraLines = o3d.geometry.LineSet.create_camera_visualization(view_width_px=image_width, view_height_px=image_height, intrinsic=standardCameraParametersObj.intrinsic.intrinsic_matrix, extrinsic=standardCameraParametersObj.extrinsic)
# vis.add_geometry(cameraLines)



vis.run()

vis.destroy_window()



[Open3D WARNING] GLFW Error: WGL: Failed to make context current: 控制代碼無效。 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: 控制代碼無效。 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: 控制代碼無效。 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: 控制代碼無效。 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: 控制代碼無效。 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: 控制代碼無效。 


In [4]:
#[[相機參數]] #上面程式只用到image_width & image_height
# Camera parameters from Gazebo
horizontal_fov =  1.5009832 # in radians 1.089
image_width = 640  # in pixels
image_height = 480  # in pixels

# Calculate the focal length
fx = image_width / (2 * np.tan(horizontal_fov / 2))
fy = fx  # Assuming square pixels, so fx == fy

# Assume cx and cy are at the center of the image
cx = image_width / 2
cy = image_height / 2

intrinsics = o3d.camera.PinholeCameraIntrinsic(image_width, image_height, fx, fy, cx, cy)
intrinsic_matrix = np.array(intrinsics)
extrinsic_matrix = np.eye(4)  # Example extrinsic matrix (identity matrix)